In [1]:
!pip install neuralforecast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.1/160.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.8/722.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.1/731.1 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 5.9 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import torch
from typing import Optional, Union, Tuple


import torch.nn.functional as F
from torch.distributions import Distribution
from torch.distributions import Bernoulli, Normal, StudentT, Poisson, NegativeBinomial

from torch.distributions import constraints


from neuralforecast import NeuralForecast
from neuralforecast.models import TFT
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.utils import AirPassengers, AirPassengersPanel, AirPassengersStatic
from neuralforecast.losses.numpy import rmse, mape
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from neuralforecast.losses.pytorch import MSE

In [3]:
from numpy.random import seed
from random import randrange

In [4]:
df = pd.read_excel("SP100_vol.xlsx", index_col="Date",parse_dates=True)
columns=df.columns
df_static=pd.read_excel("Static_df.xlsx")
n_series=len(df.columns)
train = df.iloc[:int(len(df['AMT'])*0.7)]
validation_length=int(len(df['AMT'])*0.7)-int(len(df['AMT'])*0.5)
train=train.unstack().reset_index(name="Value").rename(columns={"level_0":"unique_id", "Date":"ds", "Value":"y"})

In [5]:
df.columns

Index(['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
      dtype='object')

In [6]:
n_inputs = [3,5,10,15,21,42,84]
hidden_size = [50,100,150,200,300,500]
epochs=[50,100,150,250,350,450,550,650,750]
dropout=[0,0.2,0.3,0.4]
n_head=[2,4,5,8,10,20]
attn_dropout=[0,0.2,0.3,0.4]
learning_rate=[0.0005,0.0001,0.00005,0.00001]
num_lr_decays=[5,3,2,1]
scaler_type=["robust","standard",'minmax']

# Hyperparameter Search

In [ ]:
for m in range (5):
  i=randrange(7)
  h=randrange(6)
  a=randrange(4)
  a1=randrange(4)
  a2=randrange(4)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)

  model=TFT(h=10, input_size=n_inputs[i], stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       hidden_size=hidden_size[h],
       n_head=n_head[h], attn_dropout=attn_dropout[a1], dropout=dropout[a2], loss=MSE(),
       max_steps=epochs[k], learning_rate=learning_rate[n],
       num_lr_decays=num_lr_decays[n],
       step_size=10,
       scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=10,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%10,step_size=10)
  forecasts = forecasts.dropna()
  Y_hat=forecasts["TFT"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | Hidden Size: {hidden_size[h]} | Scaler Type: {scaler_type[o]}")
  print(f"N head: {n_head[h]} | Learning Rate: {n} | Attn Droupout: {attn_dropout[a1]} | Droupout: {dropout[a2]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE}")

INFO:lightning_fabric.utilities.seed:Global seed set to 95638978


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Epoch: 150 | Inputs: 21 | Hidden Size: 200 | Scaler Type: robust
N head: 8 | Learning Rate: 3 | Attn Droupout: 0.2 | Droupout: 0
RMSE: 0.003547927972435724 | QLIKE: 0.04647702216313569 | MAE: 0.002314909246374723


In [7]:
for m in range (5):
  i=randrange(7)
  h=randrange(6)
  a=randrange(4)
  a1=randrange(4)
  a2=randrange(4)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)

  model=TFT(h=10, input_size=n_inputs[i], stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       hidden_size=hidden_size[h],
       n_head=n_head[h], attn_dropout=attn_dropout[a1], dropout=dropout[a2], loss=MSE(),
       max_steps=epochs[k], learning_rate=learning_rate[n],
       num_lr_decays=num_lr_decays[n],
       step_size=10,
       scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=10,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%10,step_size=10)
  forecasts = forecasts.dropna()
  Y_hat=forecasts["TFT"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | Hidden Size: {hidden_size[h]} | Scaler Type: {scaler_type[o]}")
  print(f"N head: {n_head[h]} | Learning Rate: {n} | Attn Droupout: {attn_dropout[a1]} | Droupout: {dropout[a2]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE}")

INFO:lightning_fabric.utilities.seed:Global seed set to 74914868


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Epoch: 350 | Inputs: 84 | Hidden Size: 100 | Scaler Type: robust
N head: 4 | Learning Rate: 0 | Attn Droupout: 0.2 | Droupout: 0.3
RMSE: 0.0035482787640781725 | QLIKE: 0.04598206850017933 | MAE: 0.0023296411427482216


In [8]:
for m in range (5):
  i=randrange(7)
  h=randrange(6)
  a=randrange(4)
  a1=randrange(4)
  a2=randrange(4)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)

  model=TFT(h=10, input_size=n_inputs[i], stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       hidden_size=hidden_size[h],
       n_head=n_head[h], attn_dropout=attn_dropout[a1], dropout=dropout[a2], loss=MSE(),
       max_steps=epochs[k], learning_rate=learning_rate[n],
       num_lr_decays=num_lr_decays[n],
       step_size=10,
       scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=10,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%10,step_size=10)
  forecasts = forecasts.dropna()
  Y_hat=forecasts["TFT"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | Hidden Size: {hidden_size[h]} | Scaler Type: {scaler_type[o]}")
  print(f"N head: {n_head[h]} | Learning Rate: {n} | Attn Droupout: {attn_dropout[a1]} | Droupout: {dropout[a2]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE}")

INFO:lightning_fabric.utilities.seed:Global seed set to 123795780


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Epoch: 350 | Inputs: 15 | Hidden Size: 150 | Scaler Type: robust
N head: 5 | Learning Rate: 3 | Attn Droupout: 0 | Droupout: 0.2
RMSE: 0.003593290159483864 | QLIKE: 0.04697385392341414 | MAE: 0.002374988975233513


In [9]:
for m in range (5):
  i=randrange(7)
  h=randrange(6)
  a=randrange(4)
  a1=randrange(4)
  a2=randrange(4)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)

  model=TFT(h=10, input_size=n_inputs[i], stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       hidden_size=hidden_size[h],
       n_head=n_head[h], attn_dropout=attn_dropout[a1], dropout=dropout[a2], loss=MSE(),
       max_steps=epochs[k], learning_rate=learning_rate[n],
       num_lr_decays=num_lr_decays[n],
       step_size=10,
       scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=10,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%10,step_size=10)
  forecasts = forecasts.dropna()
  Y_hat=forecasts["TFT"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | Hidden Size: {hidden_size[h]} | Scaler Type: {scaler_type[o]}")
  print(f"N head: {n_head[h]} | Learning Rate: {n} | Attn Droupout: {attn_dropout[a1]} | Droupout: {dropout[a2]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE}")

INFO:lightning_fabric.utilities.seed:Global seed set to 34205429


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

...

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Epoch: 150 | Inputs: 42 | Hidden Size: 500 | Scaler Type: standard
N head: 20 | Learning Rate: 2 | Attn Droupout: 0.2 | Droupout: 0.2
RMSE: 0.0036045468506396027 | QLIKE: 0.0467897191835059 | MAE: 0.0023878134895527316


In [10]:
for m in range (5):
  i=randrange(7)
  h=randrange(6)
  a=randrange(4)
  a1=randrange(4)
  a2=randrange(4)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)

  model=TFT(h=10, input_size=n_inputs[i], stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       hidden_size=hidden_size[h],
       n_head=n_head[h], attn_dropout=attn_dropout[a1], dropout=dropout[a2], loss=MSE(),
       max_steps=epochs[k], learning_rate=learning_rate[n],
       num_lr_decays=num_lr_decays[n],
       step_size=10,
       scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=10,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%10,step_size=10)
  forecasts = forecasts.dropna()
  Y_hat=forecasts["TFT"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | Hidden Size: {hidden_size[h]} | Scaler Type: {scaler_type[o]}")
  print(f"N head: {n_head[h]} | Learning Rate: {n} | Attn Droupout: {attn_dropout[a1]} | Droupout: {dropout[a2]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE}")

INFO:lightning_fabric.utilities.seed:Global seed set to 86508526


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Epoch: 650 | Inputs: 3 | Hidden Size: 50 | Scaler Type: standard
N head: 2 | Learning Rate: 2 | Attn Droupout: 0.3 | Droupout: 0.3
RMSE: 0.004328070511138808 | QLIKE: 0.05747974135087379 | MAE: 0.0028899316888822526


In [11]:
for m in range (5):
  i=randrange(7)
  h=randrange(6)
  a=randrange(4)
  a1=randrange(4)
  a2=randrange(4)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)

  model=TFT(h=10, input_size=n_inputs[i], stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       hidden_size=hidden_size[h],
       n_head=n_head[h], attn_dropout=attn_dropout[a1], dropout=dropout[a2], loss=MSE(),
       max_steps=epochs[k], learning_rate=learning_rate[n],
       num_lr_decays=num_lr_decays[n],
       step_size=10,
       scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=10,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%10,step_size=10)
  forecasts = forecasts.dropna()
  Y_hat=forecasts["TFT"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | Hidden Size: {hidden_size[h]} | Scaler Type: {scaler_type[o]}")
  print(f"N head: {n_head[h]} | Learning Rate: {n} | Attn Droupout: {attn_dropout[a1]} | Droupout: {dropout[a2]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE}")

INFO:lightning_fabric.utilities.seed:Global seed set to 30286117


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Epoch: 350 | Inputs: 42 | Hidden Size: 300 | Scaler Type: robust
N head: 10 | Learning Rate: 3 | Attn Droupout: 0.2 | Droupout: 0
RMSE: 0.0035818231400615583 | QLIKE: 0.04685301597824929 | MAE: 0.0023342160951936967


In [12]:
for m in range (5):
  i=randrange(7)
  h=randrange(6)
  a=randrange(4)
  a1=randrange(4)
  a2=randrange(4)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)

  model=TFT(h=10, input_size=n_inputs[i], stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       hidden_size=hidden_size[h],
       n_head=n_head[h], attn_dropout=attn_dropout[a1], dropout=dropout[a2], loss=MSE(),
       max_steps=epochs[k], learning_rate=learning_rate[n],
       num_lr_decays=num_lr_decays[n],
       step_size=10,
       scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=10,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%10,step_size=10)
  forecasts = forecasts.dropna()
  Y_hat=forecasts["TFT"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | Hidden Size: {hidden_size[h]} | Scaler Type: {scaler_type[o]}")
  print(f"N head: {n_head[h]} | Learning Rate: {n} | Attn Droupout: {attn_dropout[a1]} | Droupout: {dropout[a2]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE}")

INFO:lightning_fabric.utilities.seed:Global seed set to 127603156


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Epoch: 550 | Inputs: 3 | Hidden Size: 50 | Scaler Type: robust
N head: 2 | Learning Rate: 1 | Attn Droupout: 0.2 | Droupout: 0.4
RMSE: 0.004492344051627502 | QLIKE: 0.06068242435566357 | MAE: 0.0029066910369815167


INFO:lightning_fabric.utilities.seed:Global seed set to 3197671

Epoch: 150 | Inputs: 21 | Hidden Size: 200 | Scaler Type: robust
N head: 8 | Learning Rate: 3 | Attn Droupout: 0.2 | Droupout: 0
RMSE: 0.003547927972435724 | QLIKE: 0.04647702216313569 | MAE: 0.002314909246374723


In [14]:
for m in range (5):
  randomseed=randrange(129228148)
  model=TFT(h=10, input_size=21, stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       hidden_size=200,
       n_head=8, attn_dropout=0.2, dropout=0, loss=MSE(),
       max_steps=150, learning_rate=learning_rate[3],
       num_lr_decays=num_lr_decays[3],
       step_size=10,
       scaler_type='robust',
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=10,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%10,step_size=10)
  forecasts = forecasts.dropna()
  Y_hat=forecasts["TFT"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Random Seed: {randomseed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAPE: {MAE}")

INFO:lightning_fabric.utilities.seed:Global seed set to 102184591


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

...

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Random Seed: 11481631
RMSE: 0.0035573130269314257 | QLIKE: 0.04673876171203941 | MAPE: 0.002325224727654423


In [15]:
for m in range (5):
  randomseed=randrange(129228148)
  model=TFT(h=10, input_size=21, stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       hidden_size=200,
       n_head=8, attn_dropout=0.2, dropout=0, loss=MSE(),
       max_steps=150, learning_rate=learning_rate[3],
       num_lr_decays=num_lr_decays[3],
       step_size=10,
       scaler_type='robust',
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=10,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%10,step_size=10)
  forecasts = forecasts.dropna()
  Y_hat=forecasts["TFT"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Random Seed: {randomseed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE}")

INFO:lightning_fabric.utilities.seed:Global seed set to 97976141


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

...

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Random Seed: 97907889
RMSE: 0.0035522609973833003 | QLIKE: 0.04649525121985234 | MAE: 0.0023216583508707408


In [16]:
for m in range (5):
  randomseed=randrange(129228148)
  model=TFT(h=10, input_size=21, stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       hidden_size=200,
       n_head=8, attn_dropout=0.2, dropout=0, loss=MSE(),
       max_steps=150, learning_rate=learning_rate[3],
       num_lr_decays=num_lr_decays[3],
       step_size=10,
       scaler_type='robust',
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=10,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%10,step_size=10)
  forecasts = forecasts.dropna()
  Y_hat=forecasts["TFT"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  print(f"Random Seed: {randomseed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE}")

INFO:lightning_fabric.utilities.seed:Global seed set to 31369456


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

...

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Random Seed: 122655708
RMSE: 0.0035444249036955524 | QLIKE: 0.04634172194801512 | MAE: 0.002314673914504068


# Test

In [17]:
test_length=int(len(df['AMT'])*0.3)
df1=df.unstack().reset_index(name="Value").rename(columns={"level_0":"unique_id", "Date":"ds", "Value":"y"})

In [18]:
model=TFT(h=10, input_size=21, stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
     'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
     'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
     'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
     'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
     'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
     'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
     'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
     hidden_size=200,
     n_head=8, attn_dropout=0.2, dropout=0, loss=MSE(),
     max_steps=150, learning_rate=learning_rate[3],
     num_lr_decays=num_lr_decays[3],
     step_size=10,
     scaler_type='robust',
              random_seed=20918672)
fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
forecasts = fcst.cross_validation(df=df1,val_size=10,static_df=df_static,n_windows=None, test_size=test_length-test_length%10,step_size=10)
forecasts = forecasts.dropna()
Y_hat=forecasts["TFT"].values
Y_true=forecasts["y"].values
RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE}")

INFO:lightning_fabric.utilities.seed:Global seed set to 20918672


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

RMSE: 0.005982626743825336 | QLIKE: 0.05933192264168713 | MAE: 0.00353710720518025


In [19]:
df2=forecasts.drop(columns=["cutoff", "y"]).set_index("ds")
df2=df2.pivot(columns="unique_id")

In [20]:
df2.to_excel("TFT forecasts.xlsx")